# Build a cuisine recommender

In [1]:
%pip install skl2onnx
import pandas as pd 


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [3]:
data = pd.read_csv('../data/cleaned_cuisines.csv')
data.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Check the column names in the dataset
print("Column names:")
print(data.columns.tolist())
print("\nFirst few rows:")
data.head()


In [5]:
X = data.iloc[:, 2:]
X.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
y = data[['cuisine']]   
y.head()


,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = SVC(kernel='linear', C=10, probability=True, random_state=0)
model.fit(X_train, y_train.values.ravel())

SVC(C=10, kernel='linear', probability=True, random_state=0)

In [16]:
y_pred = model.predict(X_test)

In [17]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

     chinese       0.73      0.74      0.74       244
      indian       0.87      0.87      0.87       240
    japanese       0.76      0.75      0.75       241
      korean       0.84      0.78      0.81       235
        thai       0.77      0.84      0.80       239

    accuracy                           0.79      1199
   macro avg       0.80      0.80      0.80      1199
weighted avg       0.80      0.79      0.79      1199



In [19]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

In [21]:
onx = convert_sklearn(model, initial_types=initial_type, options= options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())